In [1]:
import cv2
import sys
import numpy as np
import matplotlib.pyplot as plt
import csv
import math
import os


# ビデオデータの読み込み
filename ="C:\\Users\\flow\\Desktop\\zemi\\Balmforth_JNNFM_16062021.mp4"
# 動画の読み込み
cap = cv2.VideoCapture(filename=filename)
#動画ファイルの情報を取得する
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
hight = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
total_frame = cap.get(cv2.CAP_PROP_FRAME_COUNT)
fps = cap.get(cv2.CAP_PROP_FPS)

print("動画の幅: ", width)
print("動画の高さ: ", hight)
print("総フレーム数: ", total_frame)
print("fps: ", fps)
print("動画の長さ: ", total_frame/fps, "[sec]")


動画の幅:  1920.0
動画の高さ:  1080.0
総フレーム数:  134927.0
fps:  25.0
動画の長さ:  5397.08 [sec]


In [2]:
#フレーム数で画像切り抜くよ
#dir_path='C:\\Users\\konko\\OneDrive\\zemi\\frame'
def save_frame_range(video_path, start_frame, stop_frame, step_frame,dir_path, basename, ext='jpg'):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        return

    os.makedirs(dir_path, exist_ok=True)
    base_path = os.path.join(dir_path, basename)

    digit = len(str(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))))

    for n in range(start_frame, stop_frame, step_frame):
        cap.set(cv2.CAP_PROP_POS_FRAMES, n)
        ret, frame = cap.read()
        if ret:
            cv2.imwrite('{}_{}.{}'.format(base_path, str(n).zfill(digit), ext), frame)
        else:
            return

save_frame_range('C:\\Users\\flow\\Desktop\\zemi\\Balmforth_JNNFM_16062021.mp4',0, 97000, 450,'C:\\Users\\flow\\Desktop\\zemi\\frame\\', 'sample_video_img')


In [3]:
#パワポ変換するよ
import glob
import re
from pptx import Presentation
from glob import glob
from PIL import Image
import os

#スライドサイズ
#4:3 (default) 9144000x6858000
#16:9 12193200x6858000
SLIDE_WIDTH, SLIDE_HEIGHT = 12193200, 6858000
#スライド中心のX、Y座標（左上が原点）
IMG_CENTER_X, IMG_CENTER_Y = SLIDE_WIDTH/2, SLIDE_HEIGHT/2
#スライドのアスペクト比
SLIDE_ASPECT_RATIO = SLIDE_WIDTH / SLIDE_HEIGHT

#出力ファイル名
OUTPUT_FILE_PATH = "zemi1.pptx"
#画像の格納ディレクトリ
IMG_DIR = "frame/"

#受け取ったプレゼンテーションオブジェクトにスライドを追加し、追加されたスライドオブジェクトを返す。
def add_slide(prs):
  #白紙スライドの追加(ID=6は白紙スライド)
  blank_slide_layout = prs.slide_layouts[6]    
  slide = prs.slides.add_slide(blank_slide_layout)
  return slide

#画像をスライド中心に貼り付ける
def add_picture(slide, img_file):
  #画像サイズを取得してアスペクト比を得る
  im = Image.open(img_file)
  im_width, im_height = im.size
  aspect_ratio = im_width/im_height

  #スライドと画像のアスペクト比に応じて処理を分岐
  #画像のほうが横長だったら横めいっぱいに広げる
  if aspect_ratio > SLIDE_ASPECT_RATIO:
    img_display_width = SLIDE_WIDTH
    img_display_height = img_display_width / aspect_ratio
  else: #画像のほうが縦長だったら縦めいっぱいに広げる
    img_display_height = SLIDE_HEIGHT
    img_display_width = img_display_height * aspect_ratio
  #センタリングする場合の画像の左上座標を計算
  left = IMG_CENTER_X - img_display_width / 2
  top = IMG_CENTER_Y - img_display_height / 2

  #画像をスライドに追加
  if aspect_ratio > SLIDE_ASPECT_RATIO:
    slide.shapes.add_picture(img_file, left, top, width = img_display_width)  
  else:
    slide.shapes.add_picture(img_file, left, top, height = img_display_height)  

  return slide


#スライドオブジェクトの定義
prs = Presentation()
#スライドサイズの指定
prs.slide_width = SLIDE_WIDTH
prs.slide_height = SLIDE_HEIGHT

#img画像のファイル名を取得
img_files = os.listdir(IMG_DIR)
#pngで終了するファイル名のみ抽出。貼り付けたい画像の拡張子に応じて変える
img_files = [name for name in img_files if name.endswith(".jpg")]
#昇順にソート（この順番でスライドに貼り付けられる）
img_files.sort()#昇順にsort

for name in img_files:
  path = IMG_DIR + name
  slide = add_slide(prs)
  add_picture(slide, path)
#pptxファイルを出力する
prs.save(OUTPUT_FILE_PATH)